In [1]:
#!conda install -c conda-forge pandas pandas-profiling matplotlib openpyxl -y

In [2]:
#!pip install pycaret 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('clean_churn_data.csv',index_col='Unnamed: 0')

In [3]:
#from pycaret.classification import * 

In [4]:
from pycaret.classification import setup, compare_models, predict_model, save_model, load_model

In [5]:
X = df.drop(["customerID", "Box_Cox",'TotalCharges_Transform','TotalChargeslog/MonthlyCharges_Ratio','MonthlyCharges/TotalCharges_Ratio','Tenuresqrt/TotalCharges_Ratio'], axis = "columns", inplace = True)

In [6]:
automl = setup(df, target='Churn', numeric_features = ['Contract', 'PaymentMethod', 'PhoneService'])

,Description,Value
0,session_id,8225
1,Target,Churn
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(7032, 7)"
5,Missing Values,False
6,Numeric Features,6
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [7]:
#?setup

In [8]:
automl

('lightgbm',
 None,
 False,
 {'lr': <pycaret.containers.models.classification.LogisticRegressionClassifierContainer at 0x7ffda6823a90>,
  'knn': <pycaret.containers.models.classification.KNeighborsClassifierContainer at 0x7ffda6823be0>,
  'nb': <pycaret.containers.models.classification.GaussianNBClassifierContainer at 0x7ffda6823c40>,
  'dt': <pycaret.containers.models.classification.DecisionTreeClassifierContainer at 0x7ffda6823e20>,
  'svm': <pycaret.containers.models.classification.SGDClassifierContainer at 0x7ffda6823e50>,
  'rbfsvm': <pycaret.containers.models.classification.SVCClassifierContainer at 0x7ffda6827400>,
  'gpc': <pycaret.containers.models.classification.GaussianProcessClassifierContainer at 0x7ffda6827520>,
  'mlp': <pycaret.containers.models.classification.MLPClassifierContainer at 0x7ffda68275e0>,
  'ridge': <pycaret.containers.models.classification.RidgeClassifierContainer at 0x7ffda6827880>,
  'rf': <pycaret.containers.models.classification.RandomForestClassifier

In [9]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7934,0.8341,0.5135,0.6503,0.5727,0.4390,0.4451,0.1610
lr,Logistic Regression,0.7901,0.8313,0.5150,0.6392,0.5699,0.4333,0.4380,0.7150
ada,Ada Boost Classifier,0.7887,0.8324,0.4999,0.6387,0.5596,0.4237,0.4298,0.0940
ridge,Ridge Classifier,0.7844,0.0000,0.4375,0.6514,0.5227,0.3904,0.4037,0.0090
lda,Linear Discriminant Analysis,0.7836,0.8193,0.4925,0.6271,0.5511,0.4113,0.4169,0.0150
lightgbm,Light Gradient Boosting Machine,0.7822,0.8238,0.5075,0.6179,0.5565,0.4142,0.4181,0.1890
svm,SVM - Linear Kernel,0.7749,0.0000,0.3899,0.6435,0.4731,0.3447,0.3670,0.0230
rf,Random Forest Classifier,0.7692,0.7982,0.4788,0.5889,0.5277,0.3773,0.3811,0.2530
knn,K Neighbors Classifier,0.7592,0.7480,0.4412,0.5686,0.4959,0.3415,0.3466,0.0280
et,Extra Trees Classifier,0.7538,0.7711,0.4796,0.5501,0.5118,0.3484,0.3502,0.2100


In [10]:
best_model

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=8225, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [11]:
df.iloc[-2:-1].shape

(1, 7)

In [12]:
predict_model(best_model, df.iloc[-2:-1])

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Label,Score
7041,4,1,0,1,74.4,306.6,1,1,0.6797


In [13]:
save_model(best_model, 'LDA')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=['Contract',
                                                           'PaymentMethod',
                                                           'PhoneService'],
                                       target='Churn', time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None...
                                             learning_rate=0.1, loss='deviance',
                                             max_depth=3, max_features=None,
                                             max_leaf_nodes=None

In [14]:
import pickle

with open('LDA_model.pk', 'wb') as f:
    pickle.dump(best_model, f)

In [15]:
with open('LDA_model.pk', 'rb') as f:
    loaded_model = pickle.load(f)

In [16]:
new_data = df.iloc[-2:-1].copy()
new_data.drop('Churn', axis=1, inplace=True)
loaded_model.predict(new_data)

array([1])

In [17]:
loaded_lda = load_model('LDA')

Transformation Pipeline and Model Successfully Loaded


In [18]:
predict_model(loaded_lda, new_data)

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Label,Score
7041,4,1,0,1,74.4,306.6,1,0.6797


In [19]:
from IPython.display import Code

Code('predict_churn.py')

import pandas as pd
from pycaret.classification import predict_model, load_model

def load_data(filepath):
    """
    Loads Churn data into a DataFrame from a string filepath.
    """
    df = pd.read_csv(filepath)
    return df


def make_predictions(df):
    """
    Uses the pycaret best model to make predictions on data in the df dataframe.
    """
    model = load_model('IDA')
    predictions = predict_model(model, data=df)
    predictions.rename({'Label': 'Churn_prediction'}, axis=1, inplace=True)
    predictions['Churn_prediction'].replace({1: 'Churn', 0: 'No churn'},
                                            inplace=True)
    return predictions['Churn_prediction']


if __name__ == "__main__":
    df = load_data('new_churn_data.csv')
    predictions = make_predictions(df)
    print('predictions:')
    print(predictions)

In [20]:
%run predict_churn.py

Transformation Pipeline and Model Successfully Loaded
predictions:
0       Churn
1    No churn
2    No churn
3    No churn
4    No churn
Name: Churn_prediction, dtype: object


For my algorithm I ended up using AUC as my metric because I thought that was the most important parameter to look at. Using AUC also gave me the best accuracy so it was a win win. Overall my model correctly  predicted churn for 4/5 of the customers in the new_churn_data set. I did try changing other metrics, but none were as accurate as this final model. 